# $\texttt{flarestack}$ tutorial

First of all, be sure to follow the instructions of the [README](https://github.com/icecube/flarestack/blob/master/README.md) on how to install `flarestack`.

## Logging
`flarestack` uses logging, so let's set the desired logging level. Typical logging levels, in order of verbosity, are `ERROR`, `WARNING`, `INFO`, `DEBUG`. If you are not familiar with python logging and/or you are more used to `print()` statements check out the [python logging HOWTO](https://docs.python.org/3/howto/logging.html).

In [1]:
import logging
logging.basicConfig(level='INFO')

# 1. Getting started

`flarestack` needs a local cache directory plus a source directory for the datasets. Normally, these are configured as environment variables (see `README`). Here, instead of using `export` from the shell, we set them from `python`.

In [2]:
import os

user = os.environ.get('USER')

directory = 'Work'

os.environ["FLARESTACK_SCRATCH_DIR"] = os.path.join('/home', user, directory) 
os.environ["FLARESTACK_DATASET_DIR"] = os.path.join('/home', user, directory, 'flarestack_datasets') 


Be aware that the first `import` statement will trigger the creation of `flarestack` directory structure (this may be indeed unexpected from an `import`).

In [3]:
from flarestack.shared import host_server
from flarestack.shared import fs_scratch_dir
from flarestack.data.icecube.ic_season import icecube_dataset_dir

INFO:flarestack.shared:Scratch Directory is: /home/lincetto/Work/flarestack__data/
INFO:flarestack.shared:Found Directory: /home/lincetto/Work/flarestack__data/
INFO:flarestack.shared:Found Directory: /home/lincetto/Work/flarestack__data/input/
INFO:flarestack.shared:Found Directory: /home/lincetto/Work/flarestack__data/storage/
INFO:flarestack.shared:Found Directory: /home/lincetto/Work/flarestack__data/output/
INFO:flarestack.shared:Found Directory: /home/lincetto/Work/flarestack__data/cluster/
INFO:flarestack.shared:Found Directory: /home/lincetto/Work/flarestack__data/input/pull_corrections/
INFO:flarestack.shared:Found Directory: /home/lincetto/Work/flarestack__data/cluster/logs/
INFO:flarestack.shared:Found Directory: /home/lincetto/Work/flarestack__data/input/catalogues/
INFO:flarestack.shared:Found Directory: /home/lincetto/Work/flarestack__data/input/acceptance_functions/
INFO:flarestack.shared:Found Directory: /home/lincetto/Work/flarestack__data/input/energy_pdf_splines/
INF

Now checking the environment configuration. The host server will be `None` if we are not running on the DESY or WIPAC clusters.

In [4]:
print(f'Running at {host_server} with the following config:\n - data directory: {icecube_dataset_dir}\n - scratch directory: {fs_scratch_dir}')

Running at None with the following config:
 - data directory: /home/lincetto/Work/flarestack_datasets/
 - scratch directory: /home/lincetto/Work/flarestack__data/


## 2. Using Flarestack Classes

Classes used in $\texttt{flarestack}$'s core functionality (e.g. `flarestack.core.energy_pdf.EnergyPDF`, `flarestack.core.minimisation.MinimisationHandler`, etc) have a class attribute `<class>.subclasses`.  
This is a dictionary with the structure `{<subclass name>: <subclass>}`.  

In [5]:
from flarestack.core.minimisation import MinimisationHandler
MinimisationHandler.subclasses

{'fixed_weights': flarestack.core.minimisation.FixedWeightMinimisationHandler,
 'large_catalogue': flarestack.core.minimisation.LargeCatalogueMinimisationHandler,
 'fit_weights': flarestack.core.minimisation.FitWeightMinimisationHandler,
 'flare': flarestack.core.minimisation.FlareMinimisationHandler}

For analyses we only have to pass a dictionary of the subclass names and corresponding parameters.  
To execute use `flarestack.cluster.submitter.Submitter`. This always works locally. For using the cluster, again, if you are running at DESY or WIPAC, you do not have to worry. We got you covered.

In [6]:
from flarestack.cluster.submitter import Submitter
Submitter.submitter_dict

{'local': flarestack.cluster.submitter.LocalSubmitter,
 'DESY': flarestack.cluster.submitter.DESYSubmitter,
 'WIPAC': flarestack.cluster.submitter.WIPACSubmitter}

## 3. Example: Point Source Sensitivity ##

Let's try to calculate the 10-year point source sensitivity for one declination.  
First we have to specify a name for the analysis.

In [7]:
name = "analyses/10yr_ps_sens_one_declination"

The input directory (with the analysis dictionaries), the output directory (plots, p-values, etc) and the cache directory (saved trials, etc) will be created accordingly. For example our plot output directory will be:

In [8]:
from flarestack.shared import plot_output_dir
plot_output_dir(name) # it is a name but also a path!


'/home/lincetto/Work/flarestack__data/output/plots/analyses/10yr_ps_sens_one_declination'

Many dataset implementations are available in `flarestack.data`. We will use the PS Tracks v3.2.

It is important to note that this is just and object-interface to the actual data set (that has to be previously made available in the designated directory).

In [9]:
from flarestack.data.icecube import ps_v003_p02

We want to inject a steady neutrino signal with a power law spectrum with $\gamma=2.5$. For other Energy or Time PDFs check `flarestack.core.energy_pdf` and `flarestack.core.time_pdf`.

This is as straight forward as:

In [10]:
injection_gamma = 2.5

injection_energy = {
    "energy_pdf_name": "power_law",
    "gamma": injection_gamma
}

injection_time = {
    "time_pdf_name": "steady"
}

inj_setup = {
    "injection_energy_pdf": injection_energy,
    "injection_sig_time_pdf": injection_time
}

We are looking for a steady signal with a power law spectrum. 
We assume the background to be constant in time.  
We want to use the "standard" point source likelihood. More likelihood implementations in `flarestack.core.llh`

In [11]:
llh_time = {
    "time_pdf_name": "steady"
}

llh_energy = {
    "energy_pdf_name": "power_law",
}

llh_time_bkg = {
    "time_pdf_name": "steady"
}

# here one can select "llh_name": "spatial" for spatial-only LLH ignoring energy and time PDFs
llh_setup = {
    "llh_name": "standard",
    "llh_energy_pdf": llh_energy,
    "llh_sig_time_pdf": llh_time,
    "llh_bkg_time_pdf": llh_time_bkg
}

We need a source catalogue. This catalogue will be a numpy array stored as a `.npy` file and we only pass the filename. For point sources the is a utility function to generate dummy sources.

In [12]:
from flarestack.utils.prepare_catalogue import ps_catalogue_name
import numpy as np

"""
This apparently innocuous function works behind the scenes to create a catalogue file, then returns its path. The numpy file contains a structured (array with named fields). We could create the array on-the-fly but having it stored is recommended and/or may be necessary afterwards.
"""
sindec = 0.5
catalogue_path = ps_catalogue_name(sindec=sindec)
print(f'Catalogue created at {catalogue_path}') # it could be better to have this logged!
cat = np.load(catalogue_path)
print(type(cat))

Catalogue created at /home/lincetto/Work/flarestack__data/input/catalogues/single_source/sindec_0.50.npy
<class 'numpy.ndarray'>


In [13]:
cat

array([(3.14159265, 0.52359878, 1., 1., 55800.4164699, 55750.4164699, 55900.4164699, 1., b'PS_dec=0.5')],
      dtype=[('ra_rad', '<f8'), ('dec_rad', '<f8'), ('base_weight', '<f8'), ('injection_weight_modifier', '<f8'), ('ref_time_mjd', '<f8'), ('start_time_mjd', '<f8'), ('end_time_mjd', '<f8'), ('distance_mpc', '<f8'), ('source_name', 'S30')])

Now we make a guess for our sensitivity.

Note: $\texttt{flarestack}$ is using its own flux unit $k$.

In [14]:
from flarestack.shared import flux_to_k, k_to_flux

print(f'k = 1. equals to a dN/dE unit of {k_to_flux(1.)} (Gev)^-1 (s)^-1 (cm)^-2')

k = 1. equals to a dN/dE unit of 1e-09 (Gev)^-1 (s)^-1 (cm)^-2


Here we know where the sensitivity should be. Because the analysis has been done before.

In [15]:
from flarestack.icecube_utils.reference_sensitivity import reference_sensitivity

factor = 3. # we will scan a flux range up to three times the known reference
ref_sens = reference_sensitivity(sindec, gamma=injection_gamma)
scale = factor * flux_to_k(ref_sens)

INFO:flarestack.icecube_utils.reference_sensitivity:Retrieving reference sensititivity data from `ic_season` module


Now we just have to put all the info into one dictionary to pass to the `MinimisationHanddler`

In [16]:
mh_dict = {
    "name": name,                               # unique name for the analysis
    "mh_name": "fixed_weights",                 # name of the MinimisationHandler subcalss
    "dataset": ps_v003_p02,                     # the neutrino dataset
    "catalogue": catalogue_path,                # path to the .npy catalogue file
    "inj_dict": inj_setup,                     # info for the Injector
    "llh_dict": llh_setup,                     # info for the LLH
    "scale": scale,                             # a guess for the sensitivity scale
    "n_trials": 10,                             # number of trials to run (background trials will be run ten times this number!)
    "n_steps": 10,                              # number of steps when injecting signal
    "allow_extrapolated_sensitivity": True      # allow extrapolation in the sensitivity calculation (here we do because we only run very few trials)
}

To execute the analysis we defined above we create a submitter instance

In [17]:
submitter = Submitter.get_submitter(
    mh_dict=mh_dict,                         # the analysis info
    use_cluster=False,                       # run it on the cluster if True
    n_cpu=1,                                 # number of LOCAL CPUs to use, NOTE: the number of cluster CPUs has to be specified in the cluster_kwargs!
    do_sensitivity_scale_estimation=False,   # make a guess of the sensitivity scale, for options check flarestack.cluster.submitter
    remove_old_results=True,                 # if you are running the analysis again and something changed, maybe you want to remove old trials?
#   **cluster_kwargs                         # keyword arguments used when running the cluster, This depends on the cluster obviously
)

print(submitter)


----- Submitter for analyses/10yr_ps_sens_one_declination -----
not using cluster 
using 1 CPUs locally
job-id: None 
no scale estimation 



Energise ......

In [18]:
submitter.analyse()

INFO:flarestack.core.minimisation:Using 'standard' LLH class


FileNotFoundError: [Errno 2] No such file or directory: '/home/lincetto/Work/flarestack_datasets/ps_tracks/version-003-p02/IC40_MC.npy'

To get the results we use the `ResultsHandler`. This will also create some plots like the sensitivity fit, bias plots, etc. in the plot directory.

In [ ]:
from flarestack.core.results import ResultsHandler
results_handler = ResultsHandler(submitter.mh_dict)

In [ ]:
print(fr'sensitivity flux: {results_handler.sensitivity:.2e} +{results_handler.sensitivity_err[1]}  -{results_handler.sensitivity_err[0]}')
print(f'reference: {reference_sensitivity(sindec)[0]}')
print(fr'sensitivity n_s: {results_handler.sensitivity * results_handler.flux_to_ns:.2e} +{results_handler.sensitivity_err[1] * results_handler.flux_to_ns}  -{results_handler.sensitivity_err[0] * results_handler.flux_to_ns}')

## 4. Example: Upper Limits

There are some really useful functions in `flarestack.cosmo`!  

Take the implementations of the IceCube diffuse flux measurements for example:

In [ ]:
from flarestack.cosmo.icecube_diffuse_flux import contours, get_diffuse_flux_contour
contours.keys()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
ax = plt.subplot(111)

for fit in contours.keys():

    best_fit, upper_butterfly, lower_butterfly, e_range = get_diffuse_flux_contour(fit)
    plt.plot(e_range, best_fit(e_range) * e_range**2, label=fit)
    plt.fill_between(e_range, upper_butterfly(e_range)* e_range**2, lower_butterfly(e_range)* e_range**2, alpha=0.3)

plt.yscale("log")
plt.xscale("log")
plt.xlabel(r"$E_{\nu}$")
plt.ylabel(r"$E_{\nu}^{2} \frac{dN}{dE}$")
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.4), ncol=2, fancybox=True, shadow=True, fontsize=12)
plt.tight_layout()
plt.show()
plt.close()

If you are interested in transients you also are at the right place!  
With your favourte transients population's rate, the flux normaisation at 1 GeV and the corresponding spectral index you can easily get the rate in a redshift shell, the neutrino flux per source at a certain redshift, the neutrino flux per redshift and the cumulatice neutrino flux.

In [ ]:
from flarestack.cosmo.neutrino_cosmology import define_cosmology_functions
from flarestack.cosmo.rates import get_rate
from astropy import units as u

frb_rate = get_rate('FRB')
frb_dummy_flux = 2e+46 / u.GeV
frb_example_gamma = 2

rate_per_z, nu_flux_per_z, nu_flux_per_source, cumulative_nu_flux = define_cosmology_functions(frb_rate, frb_dummy_flux, frb_example_gamma)

redshift = np.linspace(0.1, 4, 1000)

fig, axs = plt.subplots(4, sharex='all', figsize=[5, 12])
axs[0].plot(redshift, rate_per_z(redshift).to('yr-1').value)
axs[0].set_ylabel('rate [yr$^{-1}$]')

axs[1].plot(redshift, nu_flux_per_z(redshift).to('GeV-1 cm-2 s-1 sr-1').value)
axs[1].set_ylabel(r'$\nu$ flux per redshift [GeV$^{-1}$ cm$^{-2}$]')

axs[2].plot(redshift, nu_flux_per_source(redshift).to('GeV-1 cm-2').value)
axs[2].set_ylabel(r'$\nu$ flux per source [GeV$^{-1}$ cm$^{-2}$]')

axs[3].plot(redshift[1:-1], [i.to('1 / (cm2 GeV s sr)').value for i in cumulative_nu_flux(redshift)])
axs[3].set_ylabel(r'cumulative $\nu$ flux [GeV$^{-1}$ cm$^{-2}$]')
plt.show()
plt.close()

NOTE: The result of `cumulative_nu_flux()` is already your result for the contribution of the popultion to the diffuse flux!  

All the above is packed into one convenience function:

In [ ]:
from flarestack.cosmo.neutrino_cosmology import calculate_transient_cosmology

Let's use this to get some actual super interesting, timely results and revisit the FRB  asscociated with the galactic Magnetar SGR 1935+2154 (https://arxiv.org/abs/2005.10828).  
IceCube performed a search for neutrinos and found upper limits:
```
IceCube Limit is E^2 dN/dE = 5.2 × 10−2 GeV cm^-2 @ 1 GeV 
```
(http://www.astronomerstelegram.org/?read=13689)  \
The Magnetar is 16 kpc away (conservative). Let's assume a spectrum with $\gamma=2$

In [ ]:
from flarestack.core.energy_pdf import EnergyPDF

dist = 16 * u.kpc
atel_flux_norm_lim = 5.2 * 10**-2. * (u. GeV / u.cm**2) / u.GeV**2.

e_pdf_dict = {
    "energy_pdf_name": "power_law",
    "gamma": 2.0,
    "e_min_gev": 10.**3,
    "e_max_gev": 10.**6,
    "nu_flux_at_1_gev": atel_flux_norm_lim * 4 * np.pi * dist**2.
}

epdf = EnergyPDF.create(e_pdf_dict)

With these information it is now super straight forward to get the upper limits from a population of FRB's, that share SGR 1935+2154's properties, assuming they are all standard candels:

In [ ]:
fit = "joint_15"
integrated_nu_flux_1_gev = calculate_transient_cosmology(e_pdf_dict, frb_rate, "frb_limit", zmax=8.0, diffuse_fit=fit)

Let's plot our very interesting findings so we can publish in a prestegious journal!

In [ ]:
best_fit, upper_butterfly, lower_butterfly, e_range = get_diffuse_flux_contour(fit=fit)


plt.figure()
plt.plot(e_range, best_fit(e_range) * e_range**2, label="IceCube Diffuse Flux")
plt.fill_between(e_range, upper_butterfly(e_range)* e_range**2, lower_butterfly(e_range)* e_range**2, alpha=0.3)
x = [epdf.e_min, np.exp(0.5*(np.log(epdf.e_min) + np.log(epdf.e_max))), epdf.e_max]
y = np.array([integrated_nu_flux_1_gev.value for _ in range(3)]) 
plt.errorbar(x, y, yerr=0.25*y, uplims=True, label='limit')
plt.yscale("log")
plt.xscale("log")
plt.xlabel(r"$E_{\nu} [GeV] $")
plt.ylabel(r"$E_{\nu}^{2} \frac{dN}{dE}$ [GeV cm$^{-2}$ s$^{-1}$ sr$^{-1}$]")
plt.legend()
plt.show()
plt.close()